In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import h5py
import collections
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import dask
import dask.dataframe as dd

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## 1. Do citations overstate localization?
### 1.1 Compare localization rates for lawyers who operate in 1 city vs 1+ cities

In [74]:
ts = pd.read_pickle("DataStore/2018-07-P2/targ_samp_0717.pkl")
# Drop duplicates
dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# Get relevant US Patents
ts = ts.loc[~ts["tp"].isin(dup_pats)]

ts = ts[["tp", "tp_primclass", "tp_gyear", "tp_inv_msa", "tp_match_5", "tp_match_10", "cp_match_5", "cp_match_10"]]

# ts = ts[['tp', 'tp_primclass', 'tp_gyear', 'tp_inv_msa', 'tp_match_5',
#        'tp_match_10', 'year_group', 'tp_lawyer']]

In [76]:
ts.head()

,tp,tp_primclass,tp_gyear,tp_inv_msa,tp_match_5,tp_match_10,cp_match_5,cp_match_10
3,5241648,1.0,1993,"San Jose-Sunnyvale-Santa Clara, CA",0.200000,0.285714,0.064516,0.151261
5,6665662,1.0,2003,"San Francisco-Oakland-Fremont, CA",0.000000,0.145833,0.500000,0.600000
9,5797117,1.0,1998,"Los Angeles-Long Beach-Santa Ana, CA",0.000000,0.000000,0.000000,0.025641
11,7502777,1.0,2009,"San Jose-Sunnyvale-Santa Clara, CA",0.000000,0.000000,0.000000,0.000000
15,7599939,1.0,2009,"San Jose-Sunnyvale-Santa Clara, CA",0.166667,0.250000,0.150000,0.103448


In [89]:
# Limit to lawyers of patents within the sample
%time ldf = pd.read_csv("RawData/Cleaned/patent_lawyer.csv")
samp_lawyers = ldf.loc[ldf["patent"].isin(ts["tp"]), "lawyer_id"].unique()
ldf = ldf.loc[ldf["lawyer_id"].isin(samp_lawyers)]

# Lawyer for each patent
%time l2 = ldf.drop_duplicates("patent")
%time l2 = dict(zip(l2["patent"], l2["lawyer_id"]))

# Add to patents
ts["tp_lawyer"] = ts["tp"].map(l2)
del(l2)

# Location of each patent
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
.to_pandas(["patent", "inv_msa"])

# Location of each lawyer's patent
ldf["inv_msa"] = ldf["patent"].map(dict(zip(pdf["patent"], pdf["inv_msa"])))

# Drop missing locations
ldf = ldf.dropna(subset=["inv_msa"])

CPU times: user 7.08 s, sys: 335 ms, total: 7.42 s
Wall time: 6.56 s
CPU times: user 2.89 s, sys: 121 ms, total: 3.01 s
Wall time: 2.64 s
CPU times: user 857 ms, sys: 70.1 ms, total: 927 ms
Wall time: 922 ms


In [97]:
# Unique locations by lawyer
%time lawyer_loc = ldf.groupby('lawyer_id', as_index=False).agg({'inv_msa': 'unique'})
# Number of unique cities
%time num_lawyer_loc = dict(zip(lawyer_loc["lawyer_id"], [len(i) for i in lawyer_loc["inv_msa"].tolist()]))
# Location
lawyer_loc = dict(zip(lawyer_loc["lawyer_id"], lawyer_loc["inv_msa"]))

CPU times: user 12.7 s, sys: 54.4 ms, total: 12.8 s
Wall time: 12.7 s
CPU times: user 20.9 ms, sys: 0 ns, total: 20.9 ms
Wall time: 20.7 ms


In [98]:
ts["tp_lawyer_loc"] = ts["tp_lawyer"].map(lawyer_loc)
ts["num_tp_lawyer_loc"] = ts["tp_lawyer"].map(num_lawyer_loc)

In [88]:
ts[["num_tp_lawyer_loc", "tp_match_10"]].groupby("num_tp_lawyer_loc").mean()[:10]

,tp_match_10
num_tp_lawyer_loc,
1.0,0.120746
2.0,0.119691
3.0,0.125865
4.0,0.116938
5.0,0.120910
6.0,0.113576
7.0,0.116480
8.0,0.123766
9.0,0.122442


Doesn't appear that lawyers operating in multiple cities have a huge effect on rates of local citation; i.e. all lawyers cite locally a lot. It's more to do with the control of selecting on the *same* lawyer.

#### 1.1.2 How well does citations overlap with lawyer's locations?

In [158]:
all_p = ts["tp"].tolist()

%time cit = fastparquet.ParquetFile("RawData/Cleaned/cit_0628.parq").to_pandas()
c2 = cit.loc[cit["cited"].isin(all_p)]
del(cit)

%time asgs = pickle.load(open("RawData/Cleaned/patent_assignee_dict_0628.pkl", "rb"))
# Remove self-citations
%time asg_match = (set(asgs.get(cited, [])).intersection(asgs.get(citing, [])) for cited, citing \
                       in zip(c2["cited"], c2["citing"]))
%time asg_match = [len(i) for i in asg_match]
del(asgs)

c2["asg_match"] = asg_match
c2 = c2.loc[c2["asg_match"] == 0]
c2 = c2[["citing", "cited"]]

# Patent locations
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
.to_pandas(["patent", "inv_msa"])

# Location of each lawyer's patent
c2["citing_inv_msa"] = c2["citing"].map(dict(zip(pdf["patent"], pdf["inv_msa"])))
c3 = c2.dropna(subset=["citing_inv_msa"])
%time c3 = c3.groupby("cited")["citing_inv_msa"].apply(list)

CPU times: user 17.4 s, sys: 3.92 s, total: 21.4 s
Wall time: 19.3 s
CPU times: user 24.2 s, sys: 1.73 s, total: 25.9 s
Wall time: 25.9 s
CPU times: user 541 ms, sys: 247 ms, total: 788 ms
Wall time: 783 ms
CPU times: user 31.7 s, sys: 72.5 ms, total: 31.7 s
Wall time: 31.7 s
CPU times: user 31.2 s, sys: 62.6 ms, total: 31.3 s
Wall time: 31.3 s


In [160]:
# Lawyer for each citing patent
# Limit to lawyers of patents within the sample
%time ldf = pd.read_csv("RawData/Cleaned/patent_lawyer.csv")
samp_lawyers = ldf.loc[ldf["patent"].isin(c2["citing"]), "lawyer_id"].unique()
ldf = ldf.loc[ldf["lawyer_id"].isin(samp_lawyers)]

# Lawyer for each patent
%time l2 = ldf.drop_duplicates("patent")
%time l2 = dict(zip(l2["patent"], l2["lawyer_id"]))

# Add to patents
c2["citing_lawyer"] = c2["citing"].map(l2)
c4 = c2.dropna(subset=["citing_inv_msa"])
%time c4 = c4.groupby("cited")["citing_lawyer"].apply(list)
del(ldf, l2)

CPU times: user 7.47 s, sys: 203 ms, total: 7.68 s
Wall time: 6.82 s
CPU times: user 2.98 s, sys: 117 ms, total: 3.1 s
Wall time: 2.73 s
CPU times: user 873 ms, sys: 50.2 ms, total: 923 ms
Wall time: 919 ms
CPU times: user 32.4 s, sys: 50.7 ms, total: 32.5 s
Wall time: 32.4 s


In [161]:
# Get citation locations
ts["tp_cit_loc"] = ts["tp"].map(c3)

# Get citing lawyer
ts["tp_cit_lawyer"] = ts["tp"].map(c4)
del(c3, c4)

ts = ts.dropna(subset=["tp_cit_loc", "tp_lawyer", "tp_cit_lawyer"])

def tokenize(l, filter_list):
    for item in l:
        if item in filter_list:
            yield item
            
# Get lawyer locations
%time cit_in_lawyer = [len(list(tokenize(tp_c, l_l))) for tp_c, l_l\
                       in zip(ts["tp_cit_loc"].tolist(), ts["tp_lawyer_loc"].tolist())]
ts["cit_in_lawyer_loc"] = cit_in_lawyer
ts["pct_cit_in_lawyer_loc"] = ts["cit_in_lawyer_loc"]/ts["tp_cit_loc"].apply(len)

# Get lawyer match
%time lawyer_match = [len([i for i in j if i == tp_l])/len(j) for j, tp_l in zip(ts["tp_cit_lawyer"].tolist(), ts["tp_lawyer"].tolist())]
ts["pct_cit_lawyer_match"] = lawyer_match

CPU times: user 1min 16s, sys: 29.5 ms, total: 1min 16s
Wall time: 1min 16s
CPU times: user 979 ms, sys: 922 µs, total: 980 ms
Wall time: 979 ms


In [167]:
ts[["pct_cit_in_lawyer_loc", "pct_cit_lawyer_match"]].describe()

,pct_cit_in_lawyer_loc,pct_cit_lawyer_match
count,255236.000000,255236.000000
mean,0.475913,0.016455
std,0.387421,0.099520
min,0.000000,0.000000
25%,0.045455,0.000000
50%,0.475000,0.000000
75%,0.886957,0.000000
max,1.000000,1.000000


In [172]:
ts.to_pickle("DataStore/2018-11/jth_rep_lawyer_loc_1209.pkl")

## 2. Why localization in citations does not imply localization in innovation

### 2.1 Plenty of similar inventions in different cities

I found lists of 50 closest neighbours here: https://sfengc7.stern.nyu.edu:8888/notebooks/patentdiffusion/201811/Previous/1-ControlWithSim.ipynb

In [7]:
# ts_sp = pd.read_pickle("DataStore/2018-11/closest_nbrs_1105.pkl")

# Convert closest pat to integer
%time ts_sp["closest_pat"] = ts_sp["closest_pat"].apply(lambda x: x.astype(int))

print(len(ts_sp))
# Get rid of values with missing closest neighbours
ts_sp = ts_sp.loc[ts_sp["closest_pat_sim"].apply(lambda x: x[0].isnull() == False)]
ts_sp = ts_sp.reset_index(drop=True)
print(len(ts_sp))

# Check number of neighbours
print(len(ts_sp.loc[3, "closest_pat"]))

CPU times: user 7.41 s, sys: 150 ms, total: 7.56 s
Wall time: 7.54 s


- Only use patents from different assignees

In [10]:
# Patents from different assignee
asgs = pickle.load(open("RawData/Cleaned/patent_assignee_dict_0628.pkl", "rb"))
%time tp_asgs = [set(asgs.get(p, [])) for p in ts_sp["tp"]]
%time cp_asgs = [np.array([len(set(asgs.get(cp, [])).intersection(tasg)) for cp in j]) for j, tasg in zip(ts_sp["closest_pat"], tp_asgs)]
%time cp_asgs_ind = [np.where(j == 0) for j in cp_asgs]
del(asgs)

cp_cited2 = [x[i] for x,i in zip(ts_sp["closest_pat"], cp_asgs_ind)]

In [12]:
ts_sp["closest_pat_df"] = cp_cited2
ts_sp["closest_pat_diff"] = ts_sp["closest_pat"].apply(len) - ts_sp["closest_pat_df"].apply(len)
ts_sp["closest_pat_diff"].value_counts()

In [23]:
# Get their locations
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "inv_msa"])
pdf = dict(zip(pdf["patent"], pdf["inv_msa"]))
%time tp_msa = [pdf.get(p, np.nan) for p in ts_sp["tp"]]
%time cp_msa = [np.array([pdf.get(cp, np.nan) for cp in j]) for j in ts_sp["closest_pat_df"]]
%time cp_msa_match = [len(np.where(j == t)[0]) for j,t in zip(cp_msa, tp_msa)]

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


CPU times: user 2.81 s, sys: 11.8 ms, total: 2.83 s
Wall time: 2.82 s


In [29]:
ts_sp["num_cp_df_msa_match"] = cp_msa_match
ts_sp["num_cp_df"] = ts_sp["closest_pat_df"].apply(lambda x: len(x))
ts_sp["cp_df_msa_match"] = ts_sp["num_cp_df_msa_match"]/ts_sp["num_cp_df"]

print(len(ts_sp))


In [ ]:
ts_sp.to_pickle("DataStore/2018-11/tp_closest_nbrs_1209.pkl")
ts_sp.head()

#### 2.1.1 Average proportion of nearest neighbours from different firms in the same location

In [62]:
np.nanmean(ts_sp["cp_df_msa_match"])

0.03854070707450035

#### 2.1.2 Random occurence of patents from the same city

In [47]:
# What's the random occurrence of patents from the same city?
# p2 = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "inv_msa"])
p2["inv_msa_2"] = p2["inv_msa"].sample(frac=1).tolist()
p2["msa_match"] = (p2["inv_msa"] == p2["inv_msa_2"])
print(np.mean(p2["msa_match"]))

0.023703718966184697


#### 2.1.3 Overlap in citations

In [48]:
all_p = [item for sublist in ts_sp["closest_pat_df"].tolist() for item in sublist]
all_p = all_p+ts_sp["tp"].tolist()

cit = fastparquet.ParquetFile("RawData/Cleaned/cit_0628.parq").to_pandas()
c2 = cit.loc[cit["citing"].isin(all_p)]
del(cit)

asgs = pickle.load(open("RawData/Cleaned/patent_assignee_dict_0628.pkl", "rb"))
# Remove self-citations
%time asg_match = (set(asgs.get(cited, [])).intersection(asgs.get(citing, [])) for cited, citing \
                       in zip(c2["cited"], c2["citing"]))
%time asg_match = [len(i) for i in asg_match]
del(asgs)

c2["asg_match"] = asg_match
c2 = c2.loc[c2["asg_match"] == 0]
c2 = c2[["citing", "cited"]]

CPU times: user 2.06 s, sys: 1.51 s, total: 3.57 s
Wall time: 3.56 s
CPU times: user 1min 42s, sys: 173 ms, total: 1min 42s
Wall time: 1min 43s


In [55]:
# Citing patent to cited dictionary
# %time c2 = {n:g["cited"].tolist() for n,g in c2.groupby("citing")}

# Get pct of common citations
# %time tp_cite = [c2.get(tp, []) for tp in ts_sp["tp"]]
# %time pct_cite = [[len(set(c2.get(cp, [])).intersection(set(tp_c)))/len(tp_c) if len(tp_c)>0 else np.nan \
#                    for cp in j] for j, tp_c in zip(ts_sp["closest_pat_df"], tp_cite)]
del(c2)
%time pct_cite = [np.mean(l) for l in pct_cite]

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CPU times: user 5.09 s, sys: 9.55 ms, total: 5.1 s
Wall time: 5.2 s


In [61]:
# ts_sp["tp_pct_common_cited"] = pct_cite
print(np.nanmean(ts_sp["tp_pct_common_cited"]))

0.005023767850486143


In [63]:
# ts_sp.to_pickle("DataStore/2018-11/tp_closest_nbrs_1209.pkl")
# ts_sp.head()

_______
## Results tables

In [5]:
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
pdf = dict(zip(pdf["patent"], pdf["gyear"]))

In [52]:
ts = pd.read_pickle("DataStore/2018-11/jth_rep_lawyer_loc_1209.pkl")
ts_sp = pd.read_pickle("DataStore/2018-11/tp_closest_nbrs_1209.pkl")
ts2 = pd.read_pickle("DataStore/2018-07-P2/targ_samp_0717.pkl")[["tp", "tp_match_10"]]
ts_sp["tp_gyear"] = ts_sp["tp"].map(pdf)
ts_sp["tp_match_10"] = ts_sp["tp"].map(dict(zip(ts2["tp"], ts2["tp_match_10"])))
del(ts2)

def get_year_group_10(x):
    if x in range(1975,1985):
        yg = "1975-85"
    elif x in range(1985,1995):
        yg = "1985-95"
    elif x in range(1995, 2005):
        yg = "1995-05"
    elif x in range(2005,2015):
        yg = "2005-15"
    else:
        yg = np.nan
    return yg

ts_sp["year_group"] = ts_sp["tp_gyear"].apply(get_year_group_10)
ts["year_group"] = ts["tp_gyear"].apply(get_year_group_10)



In [53]:
# ts_sp.to_pickle("DataStore/2018-11/tp_closest_nbrs_1209.pkl")
# ts.to_pickle("DataStore/2018-11/jth_rep_lawyer_loc_1209.pkl")

In [ ]:
ts = pd.read_pickle("DataStore/2018-11/jth_rep_lawyer_loc_1209.pkl")
ts_sp = pd.read_pickle("DataStore/2018-11/tp_closest_nbrs_1209.pkl")

### 1. Pct citations in lawyer's cities

In [60]:
m = ts[["pct_cit_in_lawyer_loc", "year_group"]].groupby("year_group").mean()\
.rename(columns={"pct_cit_in_lawyer_loc": "Pct Cites in Lawyer's MSAs"})
s = ts[["pct_cit_in_lawyer_loc", "year_group"]].groupby("year_group").std()\
.rename(columns={"pct_cit_in_lawyer_loc": "S.D."})
c = ts[["pct_cit_in_lawyer_loc", "year_group"]].groupby("year_group").count()\
.rename(columns={"pct_cit_in_lawyer_loc": "$N$"})
m["S.D."] = s["S.D."]
m = m*100
m["$N$"] = c["$N$"]
m = np.round(m.T,2)
m.loc["$N$"] = m.loc["$N$"].astype(int).astype(str)
m = m.drop("2005-15",1)
m.columns.name = None

In [61]:
print(m.to_latex(escape=False, index=True, column_format="lccc"))

\begin{tabular}{lccc}
\toprule
{} & 1975-85 & 1985-95 & 1995-05 \\
\midrule
Pct Cites in Lawyer's MSAs &   32.55 &   39.84 &   47.03 \\
S.D.                       &   31.93 &   33.52 &    37.4 \\
$N$                        &   31546 &   54754 &   90243 \\
\bottomrule
\end{tabular}



In [62]:
m = ts_sp[["cp_df_msa_match", "year_group"]].groupby("year_group").mean()\
.rename(columns={"cp_df_msa_match": "Pct Most Similar Pats in Same MSA"})
s = ts_sp[["cp_df_msa_match", "year_group"]].groupby("year_group").std()\
.rename(columns={"cp_df_msa_match": "S.D."})
m1 = ts_sp[["tp_match_10", "year_group"]].groupby("year_group").mean()\
.rename(columns={"tp_match_10": "Pct Cites in Same MSA"})
s1 = ts_sp[["tp_match_10", "year_group"]].groupby("year_group").std()\
.rename(columns={"tp_match_10": "S.D."})

c = ts_sp[["cp_df_msa_match", "year_group"]].groupby("year_group").count()\
.rename(columns={"cp_df_msa_match": "$N$"})

m = pd.concat([m,s,m1,s1,c], axis=1)
m = np.round(m*100,2)
m["$N$"] = m["$N$"]/100
m["$N$"] = m["$N$"].astype(int).astype(str)
m = m.T
m.columns.name = None

In [63]:
print(m.to_latex(escape=False, index=True, column_format="lcccc"))

\begin{tabular}{lcccc}
\toprule
{} & 1975-85 & 1985-95 & 1995-05 & 2005-15 \\
\midrule
Pct Most Similar Pats in Same MSA &       3 &    3.22 &    4.02 &    4.41 \\
S.D.                              &    5.09 &    5.29 &    6.39 &    6.48 \\
Pct Cites in Same MSA             &    9.16 &    9.72 &   10.96 &   12.99 \\
S.D.                              &   23.62 &   22.93 &   23.34 &   27.92 \\
$N$                               &   24124 &   41341 &   69699 &   61273 \\
\bottomrule
\end{tabular}



In [64]:
m1 = ts_sp[["tp_pct_common_cited", "year_group"]].groupby("year_group").mean()\
.rename(columns={"tp_pct_common_cited": "Pct Common Cited"})
s1 = ts_sp[["tp_pct_common_cited", "year_group"]].groupby("year_group").std()\
.rename(columns={"tp_pct_common_cited": "S.D."})

c = ts_sp[["tp_pct_common_cited", "year_group"]].groupby("year_group").count()\
.rename(columns={"tp_pct_common_cited": "$N$"})

m = pd.concat([m1,s1,c], axis=1)
m = np.round(m*100,2)
m["$N$"] = m["$N$"]/100
m["$N$"] = m["$N$"].astype(int).astype(str)
m = m.T
m.columns.name = None

In [65]:
print(m.to_latex(escape=False, index=True, column_format="lcccc"))

\begin{tabular}{lcccc}
\toprule
{} & 1975-85 & 1985-95 & 1995-05 & 2005-15 \\
\midrule
Pct Common Cited &    0.35 &    0.44 &    0.56 &    0.55 \\
S.D.             &    0.83 &    1.08 &    1.96 &    2.36 \\
$N$              &   23439 &   40294 &   58529 &   59154 \\
\bottomrule
\end{tabular}



### 2. Examples

#### 2.1 Examples of control patents

In [90]:
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "title"])
pdf = dict(zip(pdf["patent"], pdf["title"]))

In [86]:
ts = pd.read_pickle("DataStore/2018-07-P2/targ_samp_0717.pkl")[["tp", "tp_primclass", "tp_gyear", "tp_inv_msa", 
                                                                "tp_match_10", "cp", "cp_match_10"]]
ts1 = pd.read_pickle("DataStore/2018-11/closest_nbr_control_1108.pkl")[["tp", "closest_pat",\
                        "closest_pat_sim", "closest_pat_match"]].rename(columns={"closest_pat_match": "closest_pat_match_10"})
ts2 = pd.read_pickle("DataStore/2018-11/jth_rep_lawyers_control_1109.pkl")[["tp", "lawyer_cp", "lawyer_cp_match_10"]]
ts = ts.merge(ts1, how="left", on="tp")
ts = ts.merge(ts2, how="left", on="tp")

In [91]:
ts["tp_title"] = ts["tp"].map(pdf)
ts["cp_title"] = ts["cp"].map(pdf)
ts["closest_pat_title"] = ts["closest_pat"].map(pdf)
ts["lawyer_cp_title"] = ts["lawyer_cp"].map(pdf)

In [93]:
ts.to_pickle("DataStore/2018-11/jth_rep_all_controls_1209.pkl")

In [99]:
ts2 = ts.dropna(subset=["tp_title","cp_title", "closest_pat_title", "lawyer_cp_title"])[["tp_title","cp_title", "closest_pat_title", "lawyer_cp_title"]]
ts2.sample(20)

,tp_title,cp_title,closest_pat_title,lawyer_cp_title
140952,Reuse of an idle paging slot of a frame in a m...,Enabling AD-HOC data communication over establ...,Platform for serving online content,Combining mobile VPN and internet protocol
98502,Card reading terminal having protective shield...,Operator work station,Uniport interface for a bar code reading instr...,Method of merchandising cutter bits and displa...
76934,System for supporting and adjusting refrigerat...,Table base construction,Horizontal refrigerator,Automatic conduit anchorage device
64304,Self-cleaning water filtration system and method,Mobile phase gradient generation microfluidic ...,System for processing polluted water,Nanostructured separation and analysis devices...
28122,Saw table with compound movement of saw,Bakery product slicing machine,Apparatus for cutting interlocking joints,Rotary cutting dies
155957,Efficient implementation of block-based transf...,Run length coding and decoding,Apparatus and method for selective attribute d...,Nosologic system of diagnosis
268099,Apparatus and method for selectively accessing...,Limiting concurrent modification and execution...,Cache-line reuse-buffer,"Pipelined microprocessor, apparatus, and metho..."
189072,Purification of polymorphic components of comp...,Liquid single reagent for air enzyme complemen...,Method of preparing and applying single strand...,Flow cytometry apparatus
134126,Vehicle inverter assembly with cooling channels,CFP mechanical platform,Motor drive architecture for high frequency AC...,Methods and apparatus for an improved PCB asse...
203795,Voice quality on a communication link based on...,Provisioning of wireless private access subscr...,Method and apparatus for providing and obtaini...,Communications handoff using an adaptive antenna


In [110]:
i = 98502
display(list(zip(ts.loc[i, ["tp", "cp", "closest_pat", "lawyer_cp"]].tolist(), ts2.loc[i].tolist())))
print(ts.loc[i, ["tp", "tp_primclass"]])

[(5265951,
  'Card reading terminal having protective shield for input port thereof'),
 (5265952, 'Operator work station'),
 (5233169.0, 'Uniport interface for a bar code reading instrument'),
 (5443310.0, 'Method of merchandising cutter bits and display case therefor')]

tp              5265951
tp_primclass        312
Name: 98502, dtype: object


#### 2.2 Examples of citations in lawyer's MSAs

In [111]:
ts2 = pd.read_pickle("DataStore/2018-11/jth_rep_lawyer_loc_1209.pkl")

In [113]:
ts2.loc[ts2["pct_cit_in_lawyer_loc"] >= 0.7].sample(5)

,tp,tp_primclass,tp_gyear,tp_inv_msa,tp_match_5,tp_match_10,cp_match_5,cp_match_10,tp_lawyer,num_tp_lawyer_loc,tp_lawyer_loc,tp_cit_loc,cit_in_lawyer_loc,pct_cit_in_lawyer_loc,tp_cit_lawyer,pct_cit_lawyer_match,year_group
1109778,4656580,703.0,1987,"Kingston, NY",0.0,0.0,0.0,0.000000,223416f11e9a916c06f8cf089d87047c,70.0,"[Norfolk, NE, Jena, LA, Miles City, MT, Hartfo...","[San Jose-Sunnyvale-Santa Clara, CA, San Jose-...",57,0.863636,"[nan, f83afa9c8dc187c3513e6ccca52b8c03, ef844d...",0.000000,1985-95
97509,4955204,62.0,1990,"San Francisco-Oakland-Fremont, CA",0.0,0.0,NaN,0.000000,5049a9bcae438751676d30baf8dade89,207.0,"[San Jose-Sunnyvale-Santa Clara, CA, Carmel-by...","[Seattle-Tacoma-Bellevue, WA, Seattle-Tacoma-B...",17,0.944444,"[5e2531ea3c81405026098b8ea02583cb, 5e2531ea3c8...",0.000000,1985-95
26571,4367565,15.0,1983,"Reno-Sparks, NV",NaN,0.0,0.0,0.000000,7332d3b5c7f04bd7a5e903839fc5b5a1,179.0,"[Missoula, MT, Denver-Aurora-Broomfield, CO, S...","[Akron, OH, Grand Rapids-Wyoming, MI, Reno-Spa...",10,0.833333,"[nan, b0aef50cbb791f78831d1a68abaa4ed3, 7332d3...",0.083333,1975-85
651141,5623492,370.0,1997,"Boulder, CO",0.0,0.0,0.0,0.125000,5dd57b5aa65ebeedd3178a30b2213489,119.0,"[Toledo, OH, Woodville, OH, Detroit-Warren-Liv...","[San Jose-Sunnyvale-Santa Clara, CA, Minneapol...",112,0.903226,"[015d4a8c5fe3f874cd569d3845d40070, 613e63765b4...",0.000000,1995-05
1174800,6230266,713.0,2001,"Boston-Cambridge-Quincy, MA-NH",0.0,0.0,0.0,0.013158,31e4ab159dc0d2e08c0189ecda77af16,117.0,"[Barnstable Town, MA, Boston-Cambridge-Quincy,...","[Riverside-San Bernardino-Ontario, CA, San Die...",62,0.953846,"[038e71433ed6a2952d8325db21106917, 038e71433ed...",0.000000,1995-05


In [117]:
display(ts2.loc[1174800, ["tp_lawyer_loc", "tp_cit_loc"]].tolist())

[array(['Barnstable Town, MA', 'Boston-Cambridge-Quincy, MA-NH',
        'Jacksonville, FL', 'Worcester, MA',
        'Portland-South Portland-Biddeford, ME', 'Provo-Orem, UT',
        'Tucson, AZ', 'Contoocook, NH',
        'San Jose-Sunnyvale-Santa Clara, CA', 'West Barnstable, MA',
        'New York-Northern New Jersey-Long Island, NY-NJ-PA',
        'San Francisco-Oakland-Fremont, CA',
        'Providence-New Bedford-Fall River, RI-MA', 'Royalston, MA',
        'Revere, MA', 'New London, NH', 'Alum Bank, PA',
        'Davenport-Moline-Rock Island, IA-IL', 'Golden Beach, FL',
        'Manchester-Nashua, NH', 'Orlando-Kissimmee-Sanford, FL',
        'Parkersburg-Marietta-Vienna, WV-OH', 'Raleigh-Cary, NC',
        'Blue Hill, ME', 'Riverside-San Bernardino-Ontario, CA',
        'Pittsburgh, PA', 'Los Angeles-Long Beach-Santa Ana, CA',
        'Las Cruces, NM', 'Seattle-Tacoma-Bellevue, WA',
        'East Hampstead, NH', 'Phoenix-Mesa-Glendale, AZ',
        'Chicago-Joliet-Naperville,